In [1]:
import numpy as np
import tensorflow as tf

path = tf.keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt'
)
text = open(path).read().lower()
print(f'Corpus length : {len(text)}\n')

Corpus length : 600893



In [2]:
type(text)

str

In [3]:
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print(f'Number of sequences : {len(sentences)}')

chars = sorted(list(set(text)))
print(f'Unique characters : {len(chars)}')

char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization ... ')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences : 200278
Unique characters : 58
Vectorization ... 


In [4]:
from tensorflow.keras import layers

model = tf.keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [5]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [6]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    seed_text = text[start_index: start_index + maxlen]
    print(f'--- Seed text : "{seed_text}"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print(f'------ Temperature : {temperature}')
        generated_text = seed_text
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
1565/1565 [==============================] - 84s 53ms/step - loss: 1.9660
--- Seed text : "the slowly ascending ranks and classes, in which,
through fo"
------ Temperature : 0.2
the slowly ascending ranks and classes, in which,
through for the sensed the same and all the sere of the success, in a schopinity and the state of the strengter the same the same the senses the fact of the say, and all the same the fact of the same the senses the fact of the same and the self-the same the serself--the same and the same and the spirit and the sere and the same and the morality and all the some some some say, the same the serses the serself
------ Temperature : 0.5
the slowly ascending ranks and classes, in which,
through for a ordance, the starding the being for the resplations the sumpless, which the strung the higher to the compleasally sympathy of a need with the even some such the world the beligionally his indifices the traded we come and alto reself--the becaute and the compressit

through formerly grundzrestly spring because arven this essigious grodiam in us fealings,--hat it. they sayer reveetly, hewenttatirnion, moral ligit, of a order feesiganith must,"--will peepoxtary out thinating period.--farlyou lieggously, the
deterive ourselves will hubmorarpation, for turmifists,
eor sist, do itly injsists, 
hing thouly confect, with for shever.
the
othery, and spircts, which sunsediedes 
epoch 5
1565/1565 [==============================] - 87s 56ms/step - loss: 1.4493
--- Seed text : "the slowly ascending ranks and classes, in which,
through fo"
------ Temperature : 0.2
the slowly ascending ranks and classes, in which,
through for the strength to the fact of the possession of the possession of the presence of the sense of the same to be an active and all the subtle any say and such a soul and the same to the philosopher and the strength to the same the strange of the superition of the profound of the same time an expressic of the strength to the any one are not a so

definetient is more with some at the heart, that it is peruort constrict"
for the selections of natury
cruelty in furkek;
on moral philosophy of all which feight good
and explany at the 
------ Temperature : 1.2
the slowly ascending ranks and classes, in which,
through for is to
oneavy
we perhaps be be. a cruestly mankindsjof; us
beay consequens taftel nothing in the want, thit--man innotions: when dif, spir-indemens; the free in much temple of tasles and rightly puritity of the whose provided tatter degred which good emusth!--sre preifed the value of wow domain of -when some hepinagh aver ile
been indi
to bole its acvusolment: for the feelie therebychent of anmit
epoch 9
1565/1565 [==============================] - 89s 57ms/step - loss: 1.3821
--- Seed text : "the slowly ascending ranks and classes, in which,
through fo"
------ Temperature : 0.2
the slowly ascending ranks and classes, in which,
through for the superficial to some time of the subtle to the strike to the higher of the s

through for shootur, perhapsdent inecthersourable pains may synthole of the philosopher, and sicked of surricages, had a, morals that it cause
rebious as for now philosopherful rekeniartal and
ruled mind
supreme perficus there is as in -the presens on the german makes
own. in knowledge of part of contrary, when if the god" least to mavable, tour look, in power that ye should
be the are artists
respecth, in 
------ Temperature : 1.2
the slowly ascending ranks and classes, in which,
through for athide provustiy of
the power it my father unstance, sweje! but awable
thousandne of like es-itself, in haves: how a damity, has conception becaus an advance and partly here." among isertoprian--set aly--hitherto ethust" and kideflermingly
with believes, of the
puiforing their time. to mality--when they have
mirary
of other from sensed emidatical long onled this uninfacerverments of the deprecis
epoch 13
1565/1565 [==============================] - 86s 55ms/step - loss: 1.3465
--- Seed text : "the

sufferin perhaps ourselves and been a possessions of himself and and contempt and say the sense and such a soul the tendencl. and be all the general and spirit and with a sort of the pressional and the lofting of the most agree of a consequently and the animating the belief the long and the ancient respec
------ Temperature : 1.0
the slowly ascending ranks and classes, in which,
through forms them and has are thought; at
the validias luxted and frated to inscerleness of stofling to the
descensive alas
first,
has wesher history and, us, promarn spirit, and not a principle and darely them fally,
from coarse matter, matter. but
which were as leaselydy
world inside
harrve. "willing from the prisonce--but who and iny o, thinking. the kind of laft and great religious sympathbes of accoun
------ Temperature : 1.2
the slowly ascending ranks and classes, in which,
through forsionlies, truth, faeth only at uliturially, so terprises petally type, that plantic within of
leads, they with the paches

...

propeal and conception and are of all profound and supposite and formull and herd in the most proper of the desire to the fact to the absolute cons
------ Temperature : 1.0
the slowly ascending ranks and classes, in which,
through for id
instance, and situility. the gatty one must rated with this
verdant sacrifice hiars. necessary we now doild to a good will france--that the actsss
acre
spirition, manifests whes something, the prisent, profound and is follow endure, as something--sickbes; and historibity of windicial, as yet from these
personal form, last need
of "triustic" to look, belongty of constant good and from finds af
------ Temperature : 1.2
the slowly ascending ranks and classes, in which,
through food it has cliguilely; with nuencties with dirmed of her different of their
intelliget of soulce, this fristicgreke, curtuerarily of
the posures of respentptem? but thy lile, togetheg, unegilfically before they
artwomed, sick amon-crind pain frought, those patimat itwward:foghhhhhi

through for the absurding in the world with the most view to still significance of the most man of the same and merely consideration of the old and instincts to the conduct to a decided and soul, even in the way and an action of the deception of the most man of the same contrary upon the contradiction of the world in the way of a condition of the constantly who not to himself that which is created in the en
------ Temperature : 1.0
the slowly ascending ranks and classes, in which,
through for un
the intellemtuneentary only is some time; he must will is even on the courition like clement
god in
the greatest men to have just lighten
disagre, "spiring,"
in the feorth of being being rilenas,
or to it, let no one of one rends always in them? on ay of people bhen" is, only and are lotings! what was alt first in every
untir, which bad is nevertheless!--and he sain, for the fact that wheres 
------ Temperature : 1.2
the slowly ascending ranks and classes, in which,
through followness of the op

through for the strength of the proper spirits, and the spirit of the spirit of the here and conscience of the most partial continue to the way, and the super-to still the strength of the most contray of the partial conscience of the proper sense of the constantly and the strength of the spirit of the most contray of the history of the disposition of the spirit, and the german proper more will to be a thing
------ Temperature : 0.5
the slowly ascending ranks and classes, in which,
through for the contempt a speciality, to an accustomed to be as he is in the stronger souls the continue from the will to his this,
as he will are in its man of the "future" at the rest of its paternal methological proper misunderstanding of pain of the whole farty, and himself, and the fact that the religion that the present of the exception of the free spirits of the hand, a strong soul for the domain of
------ Temperature : 1.0
the slowly ascending ranks and classes, in which,
through fortunath and
can re

epoch 55
1565/1565 [==============================] - 81s 52ms/step - loss: 1.2380
--- Seed text : "the slowly ascending ranks and classes, in which,
through fo"
------ Temperature : 0.2
the slowly ascending ranks and classes, in which,
through for the most part of the formalure and such a conscience of the first to the fact that the contradiction of the first the soul which has been the moral and moral and the words of the free spirits, and words of the way, and the man of the man as a man in the ascetic the moral and the sense of the most property of the same and such a strong and conscience of the most part of the moral and property of 
------ Temperature : 0.5
the slowly ascending ranks and classes, in which,
through forms of the most despaces, which is a soul for the most philosophers of the "minds" and also the calling, as the tragely and conscience of such a bad artisting them men soul for them personal honor in man in a fait according and former single vises and will to ancient

hang, plato, like innocently born---who promd in hems verw strirgeds and ghover inhultaclitied. in theme inguthate, and became for this
passions, and
the other foregqual--and they have, to just funman.mquerples.

252. him is inventificulest powerlises,
also opinion skepticism (is, a
delight and
aight, the unmery it were there seeie state
epoch 59
1565/1565 [==============================] - 82s 52ms/step - loss: 1.2345
--- Seed text : "the slowly ascending ranks and classes, in which,
through fo"
------ Temperature : 0.2
the slowly ascending ranks and classes, in which,
through forget of the subjection of the same father and science of the strength of the same and antiquity of the most souls, and the subjection of the same time and deception, and the organism of the spirit of the same and such an individual and its own sense of the spirit, the scientific spirit in the struggle and with the assume of the same and antiquity of the scientific conscience of the spirit, the sci
------ Tempe